In [6]:
import numpy as np
n_features = 'ascvd'
term_pred = 'short_term'
data_file_name = 'data_y10_ascvd'
endpt = 10
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'
work_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2'

In [3]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [4]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [7]:
#snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
snippets_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/Git/Python_code/snippets'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [ ]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [ ]:
data_full

In [ ]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/all_training_ID_outerloop_cohort_0_10.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [ ]:
data_full

In [ ]:
data_full = data_full

In [ ]:
# decide on number of dicrete times:
#halflife=13.*365.25

#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [ ]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [ ]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    # l2_final=0.0001
    l2_final=0.1
    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)



